# City street network orientations

Compare the spatial orientations of city street networks.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

In [28]:
# define the study sites as label : query
places = {'Canberra'       : 'Canberra Central, Australia',
          'Sydney'        : 'Sydney, NSW, Australia',
          'Melbourne'       : 'City of Melbourne, VIC, Australia',
          'Brisbane'     : 'Brisbane, QLD, Australia',
          'Perth'       : 'Perth, Gosnells, Kelmscott, Western Australia, Australia',
          'Adelaide'     : 'City of Adelaide, SA, Australia',
          'Hobart'        : 'Hobart, TAS, Australia',
          'Darwin'       : 'City of Darwin, NT, Australia',
          'Wagga Wagga'       : 'Wagga Wagga, NSW, Australia',
          }

In [29]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,149.192855,-35.225555,-35.347919,149.070169,"POLYGON ((149.0701695 -35.298076, 149.0702361 ...","Canberra Central, Australian Capital Territory..."
1,151.342743,-33.364148,-34.173242,150.260825,"(POLYGON ((150.260825 -33.71567, 150.271953 -3...","Sydney, NSW, Australia"
2,144.991343,-37.775450,-37.850670,144.897090,"POLYGON ((144.89709 -37.83635, 144.89722 -37.8...","City of Melbourne, Greater Melbourne, Victoria..."
3,153.468380,-27.021274,-27.656923,152.707901,"(POLYGON ((152.7079007 -27.3296849, 152.708930...","City of Brisbane, QLD, Australia"
4,116.013481,-32.061593,-32.098940,115.972825,"POLYGON ((115.9728245 -32.0869491, 115.9728921...","Perth, Gosnells, Kelmscott, Western Australia,..."
5,138.624308,-34.898827,-34.942048,138.577265,"POLYGON ((138.5772652 -34.9088691, 138.5773162...","City of Adelaide, South Australia, Australia"
6,147.341449,-42.873231,-42.891684,147.315582,"POLYGON ((147.3155815 -42.8831842, 147.3156476...","Hobart, Tasmania, Australia"
7,130.938563,-12.330060,-12.469386,130.815117,"POLYGON ((130.815117 -12.4541575, 130.815129 -...","City of Darwin, Northern Territory, Australia"
8,147.931455,-34.808118,-35.563625,146.778780,"POLYGON ((146.7787801 -35.0984051, 146.782458 ...","Wagga Wagga, NSW, Australia"


## Get the street networks and their edge bearings

In [30]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    G = ox.add_edge_bearings(G)
    
    if weight_by_length:
        # weight bearings by length (meters)
        streets = [(d['bearing'], int(d['length'])) for u, v, k, d in G.edges(keys=True, data=True)]
        city_bearings = []
        for street in streets:
            city_bearings.extend([street[0]] * street[1])
        bearings[place] = pd.Series(city_bearings)
    else:
        # don't weight bearings, just take one value per street segment
        bearings[place] = pd.Series([data['bearing'] for u, v, k, data in G.edges(keys=True, data=True)])

## Visualize it

In [39]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Calibri', 'size':26, 'weight':'bold'}
    xtick_font = {'family':'Calibri', 'size':12, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Calibri', 'size':10, 'weight':'bold', 'alpha':0.3, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [40]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # this prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = count.tolist()
    count = [count[-1]] + count[:-1]

    count_merged = []
    count_iter = iter(count)
    for count in count_iter:
        merged_count = count + next(count_iter)
        count_merged.append(merged_count)

    return np.array(count_merged)

In [41]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})
axes = [item for sublist in axes for item in sublist]

# plot each city's polar histogram
for ax, place in zip(axes, sorted(places.keys())):
    polar_plot(ax, bearings[place], title=place)

# add super title and save full image
suptitle_font = {'family':'Open Sans', 'fontsize':60, 'fontweight':'light', 'y':1.07}
fig.suptitle('City Street Network Orientations: Australia', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.gcf().savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()